# 1. Blocker Fraud Company
 <font size="2"> Dados e problema negócio hipotéticos</font>
 
A Blocker Fraud Company é uma empresa especializada na detecção de fraudes em transações financeiras feitas atravś de dispositivos móveis.

O modelo de negócio da empresa é do tipo Serviço com a monetização feita por performance do serviço prestado, ou seja, o usuário paga uma taxa fixa sobre o sucesso da detecção de fraude das transações.

## 1.1. Estratégia de expansão

A Blocker Fraud Company está em fase de expansão no Brasil e para adquirir clientes mais rapidamente, ela adotou uma estratégia muito agressiva. A estratégia funciona da seguinte forma:

1. A empresa vai receber 25% do valor de cada transação detectada verdadeiramente como fraude.
2. A empresa vai receber 5% do valor de cada transação detectada como fraude, porém a transação é verdadeiramente legítima.
3. A empresa vai devolver 100% do valor para o cliente, a cada transação detectada como legítima, porém a transação é verdadeiramente uma fraude.

Com essa estratégia agressiva a empresa assume os riscos em falhar na detecção de fraude e é remunerada na detecção assertiva das fraudes.

## 1.2. Objetivo

Criar um modelo de alta precisão e acurácia na detecção de fraudes de transações feitas através de dispositivos móveis.

### 1.2.1. Entregáveis

Modelo em produção no qual seu acesso será feito via API, ou seja, os clientes enviarão suas transações via API para que o seu modelo as classifique como fraudulentas ou legítimas.

Além disso, você precisará entregar um relatório reportando a performance e os resultados do seu modelo em relação ao lucro e prejuízo que a empresa terá ao usar o modelo que você produziu. No seu relatório deve conter as respostas para as seguintes perguntas:

1. Qual a Precisão e Acurácia do modelo?
2. Qual a Confiabilidade do modelo em classificar as transações como legítimas ou fraudulentas?
3. Qual o Faturamento Esperado pela Empresa se classificarmos 100% das transações com o modelo?
4. Qual o Prejuízo Esperado pela Empresa em caso de falha do modelo?
5. Qual o Lucro Esperado pela Blocker Fraud Company ao utilizar o modelo?


# 2. Imports

In [1]:
import joblib
import pandas as pd
import numpy as np

## 2.1. Utils functions

In [26]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
        Cria as novas features a partir dos dados de entrada
        Parameters
        ----------
        df: Pandas DataFrame

        Returns
        -------
        df: Pandas DataFrame
            Novo dataframe com as features inseridas.
    """
    df['is_high_amount'] = define_high_amount(df, 179000)
    df['is_M_customer'] = define_merchant_customer(df)
    df['is_new_balance_orig_zero'] =  define_is_balance_zero(df, 'newbalanceOrig')
    df['is_new_balance_dest_zero'] = define_is_balance_zero(df, 'newbalanceDest')
    df['diff_orig'] = difference_between_two_columns(df, 'oldbalanceOrg', 'newbalanceOrig')
    df['diff_dest'] = difference_between_two_columns(df, 'oldbalanceDest', 'newbalanceDest')
    df['is_orig_old_balance_greater_new'] = define_column_is_greater(df, 'oldbalanceOrg','newbalanceOrig')
    df['is_dest_old_balance_greater_new'] = define_column_is_greater(df, 'oldbalanceDest','newbalanceDest')
    return df

def define_high_amount(df: pd.DataFrame, value: int) -> np.array:
    is_high_amount = np.where(df['amount']>=value, 1, 0)
    return is_high_amount

def define_merchant_customer(df: pd.DataFrame) -> np.array:
    is_merchant_customer = np.where(df['nameDest'].str.contains('M'), 1, 0)
    return is_merchant_customer

def define_is_balance_zero(df: pd.DataFrame, column: str) -> np.array:
    is_column_balance_zero = np.where(df[column]==0, 1, 0)
    return is_column_balance_zero

def difference_between_two_columns(df: pd.DataFrame, left_column: str, right_column: str) -> pd.Series:
    difference_result = df[left_column]-df[right_column]
    return difference_result

def define_column_is_greater(df: pd.DataFrame, left_column: str, right_column: str) -> np.array:
    is_greater_result = np.where(df[left_column]>df[right_column], 1, 0)
    return is_greater_result

def define_final_data_to_model(df: pd.DataFrame) -> pd.DataFrame:
    cols_to_include = ['isFlaggedFraud', 'is_high_amount', 'is_M_customer', 
                   'is_new_balance_orig_zero', 'is_orig_old_balance_greater_new',
                   'is_new_balance_dest_zero', 'is_dest_old_balance_greater_new']

    columns_drop = ['nameOrig', 'nameDest']

    for col in cols_to_include:
        df[col] = df[col].astype('category')

    df.drop(columns_drop, axis=1, inplace=True)
    return df

def predict(data, model):   
    predicted = model.predict(data)    
    return int(predicted[0])

# 3. Load model

In [52]:
model = joblib.load('../models/model_cicle_1_standardscaler.pkl')

In [53]:
model

Pipeline(steps=[('enconding_rare',
                 RareLabelEncoder(variables=['type', 'isFlaggedFraud'])),
                ('enconding_cat',
                 CountFrequencyEncoder(variables=['is_new_balance_dest_zero',
                                                  'type',
                                                  'is_orig_old_balance_greater_new',
                                                  'is_M_customer',
                                                  'is_high_amount',
                                                  'is_new_balance_orig_zero',
                                                  'is_dest_old_balance_greater_new',
                                                  'isFlaggedFraud'])),
                ('enconding_num',
                 SklearnTransformerWrapper(transformer=StandardScaler(),
                                           variables=['amount', 'oldbalanceOrg',
                                                      'newbalanceOrig',
       

# 4. Data for predict

In [54]:
data = {'step':355,
       'type':'CASH_IN',
       'amount':209562.70000,
       'nameOrig':'C1909813604',
       'oldbalanceOrg': 73845.00000,
       'newbalanceOrig': 283407.70000,
       'nameDest': 'C1501798121',
       'oldbalanceDest':4345115.38000,
       'newbalanceDest':4135552.68000,
       'isFlaggedFraud':0}
data = pd.DataFrame.from_dict(data, orient='index').T

In [55]:
df_predict = define_final_data_to_model(create_features(data))

In [56]:
df_predict

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,is_high_amount,is_M_customer,is_new_balance_orig_zero,is_new_balance_dest_zero,diff_orig,diff_dest,is_orig_old_balance_greater_new,is_dest_old_balance_greater_new
0,355,CASH_IN,209562.7,73845.0,283407.7,4345115.38,4135552.68,0,1,0,0,0,-209562.7,209562.7,0,1


# 5. Predicting

In [57]:
model.predict(df_predict) 

array([0])

# 6. Conclusion

Não é o melhor forma de fazer deploy do modelo criado, porém para esse primeiro ciclo será feito dentro do notebook e posteriormente terá todo um aplicativo focado em criar a receber a requisição, processar e devolver o resultado para a aplicação requisidora.